## Тестовое задание Мосалева Максима

### Составление датасета

Рассмотрим готовые датасеты на сайте kaggle. Были найдены некоторые готовые датасеты: (загружены на github) https://github.com/mmaxmos/test_to_BHS

Также собирать данные можно собиирать: 
1) с оффициальных сайтов игровых платформ, таких как Steam, Epic Games Store, PlayStation Store, Xbox Store, Nintendo eShop и другие. Они предоставляют доступ к информации о играх, включая их название, описание, рейтинги пользователей, продажи и т.д.  

2) с различных игровых сайтов: Stopgame, Metacritic, IGN, GameSpot, Giant Bomb, где можно найти отзывы игроков, новости и рейтинги.

Для последующей работы я выбрал датасет 'appstore-game-rating-descr.csv',где есть описание, отзывы игр из appstore. Напишем нейронную сеть для предсказания оценки пользователя по 5-балльной шкале по описанию и жанру игры.

Проверим наличие пропусков в данных, а также предобработаем описание игр, удалив лишние символы и спецсимволы:

In [38]:
#ячейка для скачивания датасета
import wget 
wget.download('https://raw.githubusercontent.com/mmaxmos/test_to_BHS/main/appstore-game-rating-descr.csv')

100% [..........................................................................] 5362837 / 5362837

'appstore-game-rating-descr.csv'

In [39]:
import numpy as np
import pandas as pd
data=pd.read_csv('appstore-game-rating-descr.csv', sep=';')
print(data.shape)
data = data.iloc[:1000, :] 

(3323, 7)


In [2]:
data.isna().sum()

Name                   0
Average User Rating    0
Price                  0
Description            0
Age Rating             0
Size                   0
Genres                 0
dtype: int64

Пропусков в данных нет

In [3]:
for i in range(data.shape[0]):
    t = data.iloc[i, 3]
    q = ['.', ',', ';', '-', '=', '+', '*', '"', "'", '\\', '!', '>', '<', '@', ':', '_']
    st=''
    for j in range(0, len(t)):
        if not (t[j] in q or t[j].isdigit()):
            st = st + t[j]
    data.iloc[i,3] = st.strip('[').strip(']')
data

,Name,Average User Rating,Price,Description,Age Rating,Size,Genres
0,! Chess !,4.5,0.00,Chess King is the best Chess gamenTouch the sc...,4,50229248,"Entertainment, Strategy, Board"
1,"""Abi: A Robot's Tale""",4.0,0.99,PocketGamer Big Indie Pitch Asian Division Wi...,4,355188736,"Strategy, Puzzle"
2,"""Ada's Farm""",3.5,0.00,ALL ADAs GAMES are NOW FREE for a LIMITED TIME...,4,58807619,"Strategy, Entertainment, Simulation"
3,"""Ada's Fashion Show""",3.5,0.00,ALL ADAs GAMES are NOW FREE for a LIMITED TIME...,4,20720901,"Entertainment, Simulation, Strategy"
4,"""Ada's Fitness Center""",3.5,0.00,ALL ADAs GAMES are NOW FREE for a LIMITED TIME...,4,14889932,"Entertainment, Simulation, Strategy"
...,...,...,...,...,...,...,...
995,Decked Builder HD,4.5,5.99,Decked Builder iPad is the iPad version of the...,9,40597504,"Strategy, Entertainment, Card"
996,Decked Builder Lite,3.5,0.00,Decked Builder is the premium deck building ap...,9,50365440,"Card, Strategy, Entertainment"
997,Decks Royale for Clash Royale,4.5,0.00,Bored of playing the same deck over and over? ...,4,74777600,"Utilities, Action, Strategy"
998,Decromancer: The Battle Card RPG,4.0,0.00,AppsZoom // ucA fresh breeze of air in a foul ...,12,392741888,"Entertainment, Role Playing, Strategy"


Сохраним датасет в формате csv

In [ ]:
data.to_csv('games_dataset.csv')

### Конструкция архитектуры нейросети

Напишем класс нейросети, который будет использоваться для анализа игровых идей. Реализуем загрузку и предобработку данных из датасета с помощью метода load. Предобработка будет состоять из токенизации описаний, удаления стоп-слов, стемминга. Также по-простому сделаем мешок слов: создадим словарь и получим векторы описаний и жанров, используя CountVectorizer. 

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79061\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79061\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\79061\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\79061\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
class GameIdeaAnalyzer():
    def load(self, data):
        self.data = data
        for i in range(data.shape[0]):
            self.data.iloc[i, 3] = str(nltk.word_tokenize(self.data.iloc[i, 3]))
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
        stemmer = PorterStemmer()
        for i in range(self.data.shape[0]):
            tokens = self.data.iloc[i, 3].strip('[').strip(']').split(sep="', '")
            tokens = [word for word in tokens if word not in stop_words]
            tokens = [stemmer.stem(word) for word in tokens]
            tokens = [lemmatizer.lemmatize(word) for word in tokens]
            self.data.iloc[i,3] = str(tokens).strip('[').strip(']').strip('"')
            
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(self.data['Description'])
        d = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([self.data, d], axis=1)
        
        X = vectorizer.fit_transform(self.data['Genres'])
        genre = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([self.data, genre], axis=1)
        self.data = self.data.drop(columns=['Genres', 'Description'], axis=1)
        self.data = self.data.drop(columns=['Name', 'Size', 'Price', 'Age Rating'], axis=1)
        display(self.data)

In [121]:
GIA = GameIdeaAnalyzer()
GIA.load(data)

Реализуем обучение нейронной сети для анализа игровых идей - fit и предсказание рейтинга игры на основе входных данных - predict. Для предсказания используем регрессию.

In [5]:
from torch import nn, optim
import torch
from sklearn.metrics import mean_squared_error

class LinearRegression(nn.Module):
    def __init__(self, inputSize, outputSize):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(inputSize, outputSize)

    def forward(self, X):
        predictions = self.linear(X)
        return predictions

In [13]:
class GameIdeaAnalyzer():
    def load(self, data):
        self.data = data
        for i in range(data.shape[0]):
            self.data.iloc[i, 3] = str(nltk.word_tokenize(self.data.iloc[i, 3]))
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
        stemmer = PorterStemmer()
        for i in range(self.data.shape[0]):
            tokens = self.data.iloc[i, 3].strip('[').strip(']').split(sep="', '")
            tokens = [word for word in tokens if word not in stop_words]
            tokens = [stemmer.stem(word) for word in tokens]
            tokens = [lemmatizer.lemmatize(word) for word in tokens]
            self.data.iloc[i,3] = str(tokens).strip('[').strip(']').strip('"')
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(self.data['Description'])
        d = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([self.data, d], axis=1)
        
        X = vectorizer.fit_transform(self.data['Genres'])
        genre = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([self.data, genre], axis=1)
        self.data = self.data.drop(columns=['Genres', 'Description'], axis=1)
        self.data = self.data.drop(columns=['Name', 'Size', 'Price', 'Age Rating'], axis=1)
        
    def fit(self, X, y):
        learning_rate = 0.01
        epochs = 800
        y_tensor = torch.from_numpy(y).float()
        X_tensor = torch.from_numpy(X).float()
        
        self.model = LinearRegression(self.data.shape[1]-1, 1)
        criterion = nn.MSELoss() 
        optimizer = optim.SGD(self.model.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            optimizer.zero_grad()
            predictions = self.model(X_tensor)
            loss = criterion(predictions, y_tensor)
            loss.backward()  # get gradients
            optimizer.step() # update parameters
            if (epoch + 1) % (epochs / 10) == 0:
                print('epoch {}, loss {}'.format(epoch, loss.item()))
    
    def predict(self, X):
        X = torch.from_numpy(X).float()
        self.prediction =self.model(X)
        return self.prediction
    
    def mse(self, y):
        return mean_squared_error(y, self.prediction.detach().numpy()) #MSE
    
    def save_model(self, filepath): # для сохранения модели
        torch.save(self.model.state_dict(), filepath)

    def load_model(self, filepath): # для загрузки модели
        self.model.load_state_dict(torch.load(filepath))
        self.model.eval()
        

### Обучение и аналитика

In [14]:
GIA = GameIdeaAnalyzer()
GIA.load(data)

In [15]:
# разделим собранный датасет на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split
X = np.array(GIA.data.drop(columns=['Average User Rating'], axis=1))
y = np.array(GIA.data['Average User Rating'])
np.random.seed(52)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [16]:
GIA.fit(X_train, y_train) # обучение

C:\Users\79061\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 79, loss 0.8085914850234985
epoch 159, loss 0.47994717955589294
epoch 239, loss 0.39973118901252747
epoch 319, loss 0.3691639304161072
epoch 399, loss 0.3542478382587433
epoch 479, loss 0.3457704484462738
epoch 559, loss 0.34044232964515686
epoch 639, loss 0.33685067296028137
epoch 719, loss 0.33430325984954834
epoch 799, loss 0.3324261009693146


In [19]:
print(GIA.mse(y_test))

0.4505127548660519


Получили MSE на тестовой выборке равной 0.45.

### Задание со звездочкой

Реализовал автоматический сбор данных отзывов и оценок игроков различных игр с сайта https://stopgame.ru/games/reviews с помощью веб-скрапинга.

In [21]:
import requests
from bs4 import BeautifulSoup

In [29]:
name = []   # название игры
review = [] # отзыв об игре
marks = []  # оценка пользователя 

url = 'https://stopgame.ru/games/reviews' 
# на сайте 516 страниц с отзывами, в дальнейшем можно собрать их все, переходя 
# по страницам с помощью url+'?page=<номер страницы>' и убавляя data-key на 1 
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
for i in range(51, 32, -1):
    st ='120'+str(i)
    w = soup.find('div', class_='_reviews-grid_pifb1_978').find('div', {"data-key": st})
    name.append(w.find('div', class_='_game-info__title_l03f1_120').text)
    
    s = str(w.find('section', class_='_content_l03f1_127 _text_l03f1_174').find_all('p', class_='_text_12po9_111 _text-width_12po9_111'))
    ss=''
    i = 0
    while i < len(s):
        if s[i]=='<':
            while s[i]!='>' and i < len(s):
                i+=1
        else:
            ss+=s[i]
        i+=1

    review.append(ss)
    
    s = str(w.find('div', class_='_stars--filled_l03f1_1'))
    s.count('0 0 576 512')
    s.count('_half-star_')
    mark = s.count('0 0 576 512') + s.count('_half-star_')/2
    marks.append(mark)

In [30]:
game_reviews = pd.DataFrame(list(zip(name, review, marks)), columns=['Name', 'Review', 'Mark'])
game_reviews

,Name,Review,Mark
0,Darkwood,"[Интересно,\nчто ощущают разработчики этой игр...",2.5
1,Koumajou Remilia: Scarlet Symphony,[Koumajou Remilia: Scarlet Symphony - это реме...,4.5
2,The Witcher 3: Wild Hunt - Blood and Wine,"[Я обожаю ""Кровь и Вино"". Наслаждаюсь прогулка...",4.5
3,Middle-earth: Shadow of Mordor,"[Жил был гг да помер со своей семьей, но благо...",4.0
4,Chuchel,[Какой же очаровательный этот комочек волос! С...,4.5
5,Killzone,"[Платформа: PS3, На фоне оглушительного успеха...",3.5
6,Hot Wheels Unleashed,"[Играя в нее убедитесь, что ваша кредитка хоро...",0.0
7,Dark Souls II: Crown of the Sunken King,"[О боссах.Варг, Церах и Разорительница Гробниц...",2.0
8,Mafia III,"[Не смотря на хреновую оптимизацию, мыло, конч...",3.0
9,Barony,[Слишком душный рогалик БЕЗ ПРОГРЕССА И ПРОКАЧ...,2.5


In [ ]:
game_reviews.to_csv('games-review.csv') #сохранение получившегося датасета